In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import pandas as pd

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('rsip', 14),
        ('rsi_emap1', 9),
        ('rsi_emap2', 45),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.ntrades = 0
        self.pnl = 0

        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsip)
        self.rsi_ema1 = bt.indicators.EMA(self.rsi, period=self.params.rsi_emap1)
        self.rsi_ema2 = bt.indicators.EMA(self.rsi, period=self.params.rsi_emap2)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)
            self.ntrades += 1

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.rsi_ema1[0] > self.rsi_ema2[0] and self.rsi_ema1[-1] <= self.rsi_ema2[-1]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.rsi_ema1[0] < self.rsi_ema2[0] and self.rsi_ema1[-1] >= self.rsi_ema2[-1]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
            self.pnl = self.broker.getvalue() - self.broker.startingcash
            self.log('(RSI Period %2d, Fast EMA %2d, Slow EMA %2d) Ending Value %.2f, Trades %2d' %
                     (self.params.rsip, self.params.rsi_emap1, self.params.rsi_emap2, 
                     self.broker.getvalue(), self.ntrades), doprint=True)

## Strategy Plot

In [ ]:
class FixedSlippageAndCommisionScheme(bt.CommInfoBase):
    '''Use this for both slippage and commissions together'''
    params = (
        ('commission', 12),
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
        )

    def _getcommission(self, size, price, pseudoexec):
        return self.p.commission

In [ ]:
# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()


In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Add the Data Feed to Cerebro
in_sample_data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.adddata(in_sample_data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Alternatively, you can use:
# cerebro.broker.setcommission(commission=0.0)
# cerebro.broker.set_slippage_fixed(.00,  slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False)

## In-sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro(optreturn = False)

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
        rsip = 14,
        rsi_emap1=range(9, 15),
        rsi_emap2=range(45, 56))

# Add the Data Feed to Cerebro
in_sample_data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.adddata(in_sample_data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Run over everything
results = cerebro.run(maxcpus=1)

## Find Optimized params

In [ ]:
# Generate results list
final_results_list = []
for run in results:
    for strategy in run:
        pnl = strategy.pnl
        fast = strategy.p.rsi_emap1
        slow = strategy.p.rsi_emap2
        final_results_list.append([(fast,slow),pnl])

#Sort Results List
by_PnL = sorted(final_results_list, key=lambda x: x[1], reverse=True)

#Print results
print('Results: Ordered by Profit:')
for result in by_PnL:
    print('Fast: {}, Slow: {}, PnL: {}'.format(result[0][0], result[0][1], result[1]))
    
opt_rsi_ema1 = by_PnL[0][0][0]
opt_rsi_ema2 = by_PnL[0][0][1]

## In-sample Optimized params Plot

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
        rsip = 14,
        rsi_emap1=opt_rsi_ema1,
        rsi_emap2=opt_rsi_ema2)


# Add the Data Feed to Cerebro
in_sample_data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])
cerebro.adddata(in_sample_data)

# Add analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Months)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', results[0].analyzers.sharpe.get_analysis())
cerebro.plot(volume=False)


## Out-of-Sample Plot

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy,
        rsip = 14,
        rsi_emap1=opt_rsi_ema1,
        rsi_emap2=opt_rsi_ema2)


# Add the Data Feed to Cerebro
out_of_sample_data = bt.feeds.PandasData(dataname = df['2018-01-01':'2018-06-01'])
cerebro.adddata(out_of_sample_data)

# Add analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Months)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio:', results[0].analyzers.sharpe.get_analysis())
cerebro.plot(volume=False)
